In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [3]:
import findspark
findspark.init()

In [4]:
spark_url = 'local'

In [5]:
from pyspark.sql import SparkSession
from itertools import combinations
from pyspark.sql import Row
from pyspark.sql.functions import col, from_json, expr, explode, struct, count
from pyspark.sql.types import ArrayType, StringType
import csv

In [6]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

In [7]:
path = 'final2023.csv'
df = spark.read.csv(path, header=True, inferSchema=True)

In [22]:
df.show(5)

+---+--------------------+--------------------+--------------------+-------+--------------------+--------------------+-------------+----+
|_c0|               title|             authors|        affiliations|citedby|           mainterms|       subject_areas|    publisher|Year|
+---+--------------------+--------------------+--------------------+-------+--------------------+--------------------+-------------+----+
|  0|Graphene oxide-al...|['Boobphahom S.',...|['Metallurgy and ...|    0.0|['Alginate hydrog...|['Structural Biol...|Elsevier B.V.|2023|
|  1|Rare coordination...|['Chauhan C.', 'K...|['Institute of Na...|    0.0|['Coordination be...|['Analytical Chem...|Elsevier B.V.|2023|
|  2|Total ammonia nit...|['Satanwat P.', '...|['Chulalongkorn U...|    0.0|                  []| ['Aquatic Science']|Elsevier B.V.|2023|
|  3|Effects of microa...|['Buakaew T.', 'R...|['Chulalongkorn U...|    0.0|['Anaerobics', 'A...|['Environmental E...|Elsevier B.V.|2023|
|  4|Bioaccumulation o...|['Patcha

In [48]:
from pyspark.sql.functions import expr, size, split, regexp_replace
import ast

selected_df = df.select('authors', 'citedby')
selected_df = selected_df.withColumn("authors", split(regexp_replace(selected_df["authors"], "[\\[\\]]", ""), ","))
selected_df = selected_df.withColumn("authors", expr("array_union(authors, ARRAY())"))
selected_df = selected_df.withColumn("authors", expr("transform(authors, x -> trim(x))"))

selected_df = selected_df.withColumn("author_count", size(selected_df['authors']))
Q1, Q3 = selected_df.approxQuantile("author_count", [0.25, 0.75], 0.0)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_df = selected_df.filter(
    (selected_df['author_count'] >= lower_bound) &
    (selected_df['author_count'] <= upper_bound)
)

In [49]:
filtered_df.show()

+--------------------+-------+------------+
|             authors|citedby|author_count|
+--------------------+-------+------------+
|['Boobphahom S.',...|    0.0|           2|
|['Chauhan C.', 'K...|    0.0|           6|
|['Satanwat P.', '...|    0.0|           8|
|['Buakaew T.', 'R...|    0.0|           2|
|['Patchaiyappan A...|    0.0|           8|
|['Urumarudappa S....|    0.0|           4|
|['Jantorn P.', 'T...|    0.0|           6|
|['Pinit S.', 'Ari...|    0.0|           3|
|['Thanayutsiri T....|    0.0|           5|
|['Langsiri N.', '...|    0.0|           8|
|['Taecharungroj V...|    0.0|           2|
|['Rujiraprasert P...|    0.0|           9|
|['Vimon S.', 'Ker...|    0.0|           5|
|['Wongvitvichot W...|    0.0|           4|
|['Vongseenin S.',...|    0.0|           4|
|['Khamnil Y.', 'K...|    0.0|           3|
|['Chuaypen N.', '...|    0.0|           6|
|['Cheunwisat P.',...|    0.0|           8|
|['Pinsorn P.', 'S...|    0.0|          13|
|['Thamnium S.', '...|    0.0|  

In [50]:
filtered_df = filtered_df.withColumn("citedby", filtered_df["citedby"].cast("int"))
cleaned_df = filtered_df[filtered_df['citedby'] > 1]

In [56]:
cleaned_df.show()

+--------------------+-------+------------+
|             authors|citedby|author_count|
+--------------------+-------+------------+
|['Sereewatthanawu...|      4|           4|
|['Mahardawi B.', ...|      2|           7|
|['Umpreecha C.', ...|      2|           4|
|['Wahyuni D.K.', ...|      2|           8|
|['Nim B.', 'Rahay...|      2|           9|
|['Iksen I.', 'Wit...|      3|           8|
|['Jatuyosporn T.'...|      2|           8|
|['Wattanathamsan ...|      3|           8|
|['Khantibongse P....|      2|           2|
|['Nguyen A.H.L.',...|      2|           6|
|['Prasittisopin L...|      2|           3|
|['Xiao S.', 'Jian...|      3|           9|
|['Phumchusri N.',...|      2|           4|
|['Kingkaew E.', '...|      2|           4|
|['Jungtanasomboon...|      3|          10|
|['Hong-in P.', 'K...|      2|           6|
|['Songkram N.', '...|      2|           5|
|['Srikhao N.', 'O...|      5|           9|
|['Tanthanongsakku...|      2|           4|
|['Thwe Win T.', '...|      2|  

In [57]:
cleaned_df.count()

344

In [59]:
authors_schema = ArrayType(StringType())

# df_with_authors_array = cleaned_df.withColumn("authors", from_json(col("authors"), authors_schema))
df_with_authors_array = cleaned_df.withColumn("authors", expr("transform(authors, x -> trim(x))"))
filtered_rows = df_with_authors_array.filter(expr("size(authors) > 1"))

edges = filtered_rows.rdd.flatMap(
    lambda row: [Row(node1=a, node2=b) for a, b in combinations(row["authors"], 2)]
)

edges_df = spark.createDataFrame(edges)
edges_weighted = edges_df.groupBy("node1", "node2").count().withColumnRenamed("count", "weight")

filtered_edges = edges_weighted.filter(col("weight") > 1)
filtered_edges.printSchema()
filtered_edges.show()

root
 |-- node1: string (nullable = true)
 |-- node2: string (nullable = true)
 |-- weight: long (nullable = false)

+--------------------+------------------+------+
|               node1|             node2|weight|
+--------------------+------------------+------+
|    'Wiriyakijja P.'|       'Kerr A.R.'|     2|
|      'Aliyu A.A.A.'|       'Shinjo J.'|     2|
|         'Shinjo J.'|'Lohwongwatana B.'|     2|
|    'Niklander S.E.'|     'Jensen S.B.'|     2|
|        'Bhave V.M.'|   'Kimberly W.T.'|     2|
|'Seresirikachorn K.'|     'Alvarado R.'|     2|
|           'Zeng Z.'|          'Qin J.'|     2|
|          'Voiry D.'|         'Wang Q.'|     2|
|  'Kijpatanasilp I.'|     'Worobo R.W.'|     2|
|          'Jiang J.'|           'Wu R.'|     2|
|          'Villa A.'|        'Delli K.'|     2|
| 'Reubroycharoen P.'|      'Tsubaki N.'|     2|
|    'Piumsomboon P.'|   'Otarawanna S.'|     2|
|      'Aliyu A.A.A.'|       'Reed R.C.'|     2|
|  'Arunjaroensuk S.'|    'Kaboosaya B.'|     2|
|

In [61]:
filtered_edges_without_weight = filtered_edges.drop("weight")
filtered_edges_without_weight.count()

379

In [62]:
filtered_edges_without_weight.toPandas().to_csv('filtered_authorship_2023.csv', index=False)